In [1]:
from IPython.core.interactiveshell import InteractiveShell  #执行该代码可以使得当前nb支持多输出
InteractiveShell.ast_node_interactivity = "all" 
import numpy as np
import pandas as pd 
pd.options.display.max_rows = 8  
import requests
import parsel
from bs4 import BeautifulSoup
from lxml import etree
from time import sleep
from selenium.webdriver.chrome.service import Service
from selenium.webdriver import Chrome,ChromeOptions
from selenium.webdriver.common.by import By
import warnings
import csv
import random
import os

#忽略警告
warnings.filterwarnings("ignore")

1

In [2]:
# 导入所需的库
import requests
from urllib.parse import urljoin
from bs4 import BeautifulSoup
import os
import json

# 设置请求头
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

# 定义要提取的年份范围
years = range(2013, 2024)

# 初始化链接字典
link_dict = {}

# 确保实验一爬取数据文件夹存在
os.makedirs("./实验一爬取数据", exist_ok=True)

# 循环处理每个年份
for year in years:
    # 构造年份对应的索引页面URL
    index_url = f"https://www.cnipa.gov.cn/tjxx/jianbao/year{year}/indexy.html"
    
    try:
        # 发送HTTP请求获取索引页面内容
        response = requests.get(index_url, headers=headers)
        response.raise_for_status()  # 检查请求是否成功
    except requests.exceptions.RequestException as e:
        print(f"获取索引页面 {index_url} 失败: {e}")
        continue

    # 解析索引页面，寻找包含特定关键字的链接
    soup = BeautifulSoup(response.text, 'html.parser')
    links = soup.find_all('a', href=True)
    
    # 筛选包含特定关键字的链接
    target_keywords = ['专利申请代理状况', '专利申请、授权按IPC分类分布状况', '商标申请、注册及有效注册状况']
    target_link_pages = []
    for link in links:
        # 检查链接文本是否包含目标关键词之一
        if any(keyword in link.text for keyword in target_keywords):
            href = link['href']
            # 构造完整的链接URL
            full_url = urljoin(index_url, href)
            target_link_pages.append(full_url)
    
    # 对每个目标链接页面进行进一步处理
    link_dict[year] = []  # 初始化当年份的列表
    for target_link_page in target_link_pages:
        try:
            # 发送HTTP请求获取目标链接页面内容
            target_response = requests.get(target_link_page, headers=headers)
            target_response.raise_for_status()
        except requests.exceptions.RequestException as e:
            print(f"获取目标链接页面 {target_link_page} 失败: {e}")
            continue
        
        # 解析目标链接页面，提取其中的所有链接
        target_soup = BeautifulSoup(target_response.text, 'html.parser')
        sub_links = target_soup.find_all('a', href=True)
        
        # 过滤并整理子链接，排除包含 'indexy' 的链接
        sub_target_links = []
        for sub_link in sub_links:
            sub_href = sub_link['href']
            # 过滤掉非HTML文件的链接以及包含 'indexy' 的链接
            if sub_href.endswith('.html') and 'indexy' not in sub_href:
                # 构造完整的子链接URL
                sub_full_url = urljoin(target_link_page, sub_href)
                sub_target_links.append(sub_full_url)
        
        # 假设目标链接页面的标题作为大类名称
        # 尝试从目标链接页面中提取大类名称
        class_name = target_soup.find('title')
        if class_name:
            class_name = class_name.text.strip()
        else:
            # 如果没有标题，使用链接中的文件名作为大类名称
            class_name = os.path.basename(target_link_page).split('.')[0]
        
        # 将大类名称和子链接数组存储到链接字典中
        link_dict[year].append({
            'class_name': class_name,
            'links': sub_target_links
        })
    
    print(f"已处理年份 {year} 的链接")

# 保存链接字典到JSON文件
with open('./实验一爬取数据/link_dict.json', 'w', encoding='utf-8') as f:
    json.dump(link_dict, f, ensure_ascii=False, indent=4)

print("链接字典已保存到 ./实验一爬取数据/link_dict.json")

已处理年份 2013 的链接
已处理年份 2014 的链接
已处理年份 2015 的链接
已处理年份 2016 的链接
已处理年份 2017 的链接
已处理年份 2018 的链接
已处理年份 2019 的链接
已处理年份 2020 的链接
已处理年份 2021 的链接
已处理年份 2022 的链接
已处理年份 2023 的链接
链接字典已保存到 ./实验一爬取数据/link_dict.json


In [3]:
import json
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup

# 设置请求头，模拟浏览器访问
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/112.0.0.0 Safari/537.36'
}

failed_links = []

def crawl_and_process(year, class_name, link, table_title):
    """
    爬取并处理网页表格数据，并保存为CSV文件
    """
    # 创建对应年份的文件夹（如果不存在的话）
    folder_path = f'实验一爬取数据/{year}'
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)

    # 发送 GET 请求获取网页内容
    response = requests.get(link, headers=headers)
    # 设置正确的编码格式
    response.encoding = 'utf-8'
    # 使用 BeautifulSoup 解析 HTML 内容
    soup = BeautifulSoup(response.text, 'html.parser')
    # 查找所有表格
    tables = soup.find_all('table')
    
    # 检查是否存在表格
    if not tables:
        print(f"No tables found in {link}")
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "No tables found"})
        return
    
    # 尝试获取第二个表格（根据之前的代码逻辑）
    specified_tables = tables[1] if len(tables) > 1 else tables[0]
    
    # 将表格转换为 DataFrame
    try:
        df = pd.read_html(str(specified_tables))[0]
    except Exception as e:
        print(f"Failed to parse table in {link}: {e}")
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": str(e)})
        return
    
    # 对表格进行数据处理
    if len(df) > 4:
        df = df[4:]
        # 检查索引是否存在，避免越界错误
        indices_to_drop = [idx for idx in [4, 6] if idx in df.index]
        df.drop(indices_to_drop, inplace=True)
    
    # 检查 DataFrame 是否为空
    if df.empty:
        print(f'No data to save for {class_name}-{table_title}')
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "Empty DataFrame"})
        return
    
    # 检查列数是否一致
    if len(df.columns) < 1:
        print(f'No columns found in DataFrame for {class_name}-{table_title}')
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "No columns found"})
        return
    
    # 尝试进行更复杂的数据处理
    try:
        # 设置第一列作为索引
        df.set_index(df.columns[0], inplace=True)
        df.index.name = None
        
        # 将第一行作为列名
        df.columns = df.iloc[0]
        df = df.iloc[1:]
        df.columns.name = None
        
        # 将原有索引设置为一级索引，同时将第一列和第一行作为二级索引
        df = df.set_index(df.columns[0], append=True)
        df.index.name = None  # 删除索引列的名称
        
        # 创建二级索引
        df.columns = pd.MultiIndex.from_arrays([df.iloc[0], df.columns], names=[None, None])
        df = df.iloc[1:]  # 删除原始的第一行
        df = df.swaplevel(axis=1)
        
        # 保存为 CSV 文件，如果 DataFrame 不为空
        if not df.empty:
            df.to_csv(f'{folder_path}/{class_name}-{table_title}.csv', encoding='utf-8-sig')
            print(f'Successfully saved {class_name}-{table_title}.csv')
        else:
            print(f'No data to save for {class_name}-{table_title}')
            failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": "Empty DataFrame after processing"})
    except Exception as e:
        print(f"Failed to process {class_name}-{table_title}: {e}")
        failed_links.append({"year": year, "class_name": class_name, "link": link, "table_title": table_title, "reason": str(e)})

# 加载 JSON 文件
with open(r'实验一爬取数据/link_dict.json', 'r', encoding='utf-8') as f:
    link_dict = json.load(f)

# 遍历每个年份和类别链接，进行爬取和处理
for year in link_dict:
    if link_dict[year]:  # 如果该年份的类别列表不为空
        for class_info in link_dict[year]:
            class_name = class_info["class_name"]
            for link in class_info["links"]:
                # 从链接中提取表格标题
                table_title = link.split('/')[-1].replace('.html', '')
                crawl_and_process(year, class_name, link, table_title)

# 保存处理失败的链接到 JSON 文件
with open(r'实验一爬取数据/failed_links.json', 'w', encoding='utf-8') as f:
    json.dump(failed_links, f, ensure_ascii=False, indent=4)

print(f"处理失败的链接已保存到 '实验一爬取数据/failed_links.json'")

Failed to process 专利申请代理状况-d1: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请代理状况-d2: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请代理状况-d3: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请代理状况-d4: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e1: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e2: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e3: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e4: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e5: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e6: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e7: index 0 is out of bounds for axis 0 with size 0
Failed to process 专利申请、授权按IPC分类分布状况-e8: index 0 is out of bounds for

2

In [4]:
import pandas as pd
import requests
import random
from time import sleep

url = 'https://www.baiten.cn/results/filter'

# Select Zhejiang province
province = '浙江'

df = pd.DataFrame(dtype=str)

for year in range(2000, 2025):
    params = {
        'q': 'aa:((' + province + ')) AND (pd:[' + str(year) + '0101 TO ' + str(year) + '1231])',
        'fq': '',
        'sc': '35184372088831'
    }
    
    headers = {
        'Cookie': '_uab_collina=171636084297048745836638; yunsuo_session_verify=651468e84a027b9680fb898cb3261dac; JSESSIONID=467E8A8E43574AD6ADDD2CA1AA8E598B; zlcp2024423=true; BSESSION=e2148644ecde040ae13f44b2ace4184a38f0d6b0623961ad; PD=a6e2323d0e6adfd16b0bb9a0e28457c5d0c7bf66bf1f72327d2d56e14cb82af5a5f1e3e7ae91cf98c7fa13fa30f3d1fd'
    }
    
    try:
        resp = requests.post(url=url, params=params, headers=headers)
        page_json = resp.json()
        
        # Extract patent counts by type
        cn_um = int(page_json['facetPivots']['country']['cn'].get('cn_um', 0))  # Utility model
        cn_id = int(page_json['facetPivots']['country']['cn'].get('cn_id', 0))  # Design
        cn_in = int(page_json['facetPivots']['country']['cn'].get('cn_in', 0))  # Invention
        cn_gp = int(page_json['facetPivots']['country']['cn'].get('cn_gp', 0))  # Granted invention
        cn_in_gp = int(page_json['facetPivots']['country']['cn'].get('cn_in_gp', 0))  # Published invention
        
        # Add to dataframe
        df = pd.concat([df, pd.DataFrame({
            '发明专利': cn_in,
            '发明公开专利': cn_in_gp,
            '发明授权专利': cn_gp,
            '外观专利': cn_id,
            '实用新型专利': cn_um,
            '年份': year,
            '省份': province
        }, index=[len(df)])], axis=0)
        
        print(f'Successfully collected data for {year}')
        sleep(random.randint(1,3))
        
    except Exception as e:
        print(f'Failed to collect data for {year}: {e}')
        continue


# Save to Excel
df.to_excel(f'./实验一爬取数据/佰腾网/佰腾网{province}2000-2024专利数据.xlsx', index=False)
print('Data collection completed')

Successfully collected data for 2000
Successfully collected data for 2001
Successfully collected data for 2002
Successfully collected data for 2003
Successfully collected data for 2004
Successfully collected data for 2005
Successfully collected data for 2006
Successfully collected data for 2007
Successfully collected data for 2008
Successfully collected data for 2009
Successfully collected data for 2010
Successfully collected data for 2011
Successfully collected data for 2012
Successfully collected data for 2013
Successfully collected data for 2014
Successfully collected data for 2015
Successfully collected data for 2016
Successfully collected data for 2017
Successfully collected data for 2018
Successfully collected data for 2019
Successfully collected data for 2020
Successfully collected data for 2021
Successfully collected data for 2022
Successfully collected data for 2023
Successfully collected data for 2024
Data collection completed


3（验证码获取次数已超限（8513），爬不了了）

In [6]:
import os
import requests
import time
from lxml import etree
import random
from datetime import datetime, timedelta
import pandas as pd
import json
from selenium import webdriver
from selenium.webdriver.firefox.service import Service
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def get_weibo_cookie():
    """获取微博cookie"""
    options = FirefoxOptions()
    # 移除headless模式，显示浏览器窗口
    # options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    
    service = Service(executable_path='/home/gyh/bin/firefoxdriver/geckodriver')
    driver = webdriver.Firefox(service=service, options=options)
    
    try:
        # 修改为移动版微博登录页面
        driver.get("https://weibo.cn/login/")
        time.sleep(2)
        
        print("请在浏览器中完成登录...")
        input("登录完成后按回车继续...")  # 添加手动确认步骤
        
        cookies = driver.get_cookies()
        cookie_str = '; '.join([f"{cookie['name']}={cookie['value']}" for cookie in cookies])
        return cookie_str
        
    finally:
        driver.quit()

def crawl_weibo(weibo_id, target_month):
    """抓取指定微博账号的数据"""
    output_dir = './实验一爬取数据/微博数据/'
    os.makedirs(output_dir, exist_ok=True)
    
    # 先获取新的cookie
    print("正在获取微博登录cookie...")
    cookie = get_weibo_cookie()
    
    base_url = f'https://weibo.cn/u/{weibo_id}?page='
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8',
        'Accept-Encoding': 'gzip, deflate, br',
        'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8',
        'Connection': 'keep-alive',
        'Cookie': cookie  # 使用新获取的cookie
    }
    
    data = []
    page = 1
    max_retries = 3
    
    try:
        while page <= 50:  # 限制最多爬取50页
            print(f'正在爬取第{page}页...')
            retries = 0
            while retries < max_retries:
                try:
                    response = requests.get(f'{base_url}{page}', headers=headers, timeout=10)
                    response.encoding = 'utf-8'
                    tree = etree.HTML(response.text)
                    
                    if '未找到' in response.text or '没有内容' in response.text:
                        print('没有更多内容了')
                        return
                    
                    weibo_divs = tree.xpath('//div[@class="c" and @id]')
                    if not weibo_divs:
                        print('页面未包含微博内容，可能需要重新登录')
                        return
                    
                    for div in weibo_divs:
                        try:
                            time_str = div.xpath('.//span[@class="ct"]/text()')[0].split('来自')[0].strip()
                            post_time = format_time(time_str)
                            
                            if not post_time.startswith(target_month):
                                continue
                            content = extract_full_content(div, headers)
                            
                            try:
                                like_count = div.xpath('.//a[contains(text(), "赞[")]/text()')[0][2:-1]
                                repost_count = div.xpath('.//a[contains(text(), "转发[")]/text()')[0][3:-1]
                                comment_count = div.xpath('.//a[contains(text(), "评论[")]/text()')[0][3:-1]
                                comment_link = div.xpath('.//a[contains(text(), "评论[")]/@href')[0]
                            except IndexError:
                                like_count = '0'
                                repost_count = '0'
                                comment_count = '0'
                                comment_link = ''
                            
                            comments = extract_comments(comment_link, headers) if comment_link else []
                            
                            data.append({
                                '发布时间': post_time,
                                '微博内容': content,
                                '点赞数': like_count,
                                '转发数': repost_count,
                                '评论数': comment_count,
                                '评论内容': '||'.join(comments)
                            })
                            
                            print(f"成功获取一条微博：{content[:30]}...")
                            
                        except Exception as e:
                            print(f"处理单条微博时出错: {e}")
                            continue
                    
                    break  # 成功获取数据后跳出重试循环
                    
                except requests.exceptions.RequestException as e:
                    retries += 1
                    print(f"请求失败，第{retries}次重试: {e}")
                    if retries == max_retries:
                        print(f"第{page}页爬取失败，跳过")
                        break
                    time.sleep(random.uniform(5, 10))
            
            page += 1
            time.sleep(random.uniform(3, 5))
            
    except Exception as e:
        print(f"爬取过程中出错: {e}")
    
    finally:
        if data:
            df = pd.DataFrame(data)
            output_file = os.path.join(output_dir, f'知识产权_{target_month}_微博数据.csv')
            df.to_csv(output_file, index=False, encoding='utf-8-sig')
            print(f'数据已保存到: {output_file}')
            print(f'共爬取到 {len(data)} 条微博')
        else:
            print('未获取到任何数据')

# 使用示例
if __name__ == "__main__":
    weibo_id = '6188041084'  # 浙江知识产权
    target_month = '2024-04'
    
    crawl_weibo(weibo_id, target_month)

正在获取微博登录cookie...
请在浏览器中完成登录...


WebDriverException: Message: Failed to decode response from marionette


4

In [7]:
import os
import pandas as pd
import requests
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import time
import random

# 全局配置
HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Referer': 'https://www.hebnews.cn/'
}
BASE_URL = "https://world.hebnews.cn/"  # 国际新闻板块
MAX_RETRY = 3
REQUEST_TIMEOUT = 15

def get_valid_selector(soup):
    """动态获取有效的选择器"""
    # 尝试多种选择器方案
    selectors = [
        ('div.news_list > ul > li', 'h3 a', 'div.news_time'),  # 常见结构
        ('div.news-list > div.news-item', 'a.title', 'span.time'),  # 备用方案
        ('ul.list > li', 'a', 'div.meta')  # 通用方案
    ]
    
    for list_sel, title_sel, time_sel in selectors:
        items = soup.select(list_sel)
        if len(items) > 3:  # 有效列表判断
            return (list_sel, title_sel, time_sel)
    return (None, None, None)

def parse_news_list(url):
    """解析新闻列表页"""
    news_items = []
    retry = 0
    
    while retry < MAX_RETRY:
        try:
            response = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT, verify=False)
            response.encoding = 'utf-8'
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, 'lxml')
                list_sel, title_sel, time_sel = get_valid_selector(soup)
                
                if not list_sel:
                    print("未找到有效的列表选择器")
                    return []
                
                for item in soup.select(list_sel):
                    try:
                        title_elem = item.select_one(title_sel)
                        time_elem = item.select_one(time_sel) if time_sel else None
                        
                        if not title_elem:
                            continue
                            
                        news_url = urljoin(url, title_elem['href'])
                        title = title_elem.get_text(strip=True)
                        publish_time = time_elem.get_text(strip=True) if time_elem else '未知时间'
                        
                        # URL格式验证
                        if '/content_' in news_url and news_url.endswith('.htm'):
                            news_items.append({
                                'title': title,
                                'time': publish_time,
                                'url': news_url
                            })
                    except Exception as e:
                        print(f"解析条目异常: {str(e)[:50]}")
                return news_items
        except requests.exceptions.RequestException as e:
            print(f"请求失败，正在重试({retry+1}/{MAX_RETRY}): {str(e)[:50]}")
            retry += 1
            time.sleep(2**retry + random.random())
    
    print(f"无法获取列表页: {url}")
    return []

def parse_news_detail(url):
    """解析新闻详情页"""
    try:
        response = requests.get(url, headers=HEADERS, timeout=REQUEST_TIMEOUT)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'lxml')
        
        # 提取核心内容
        content = soup.select_one('div.article-content') or soup.select_one('div.content')
        if content:
            # 清理无关元素
            for elem in content.select('script, style, div.ads'):
                elem.decompose()
            content_text = content.get_text('\n', strip=True)
        else:
            content_text = "内容解析失败"
            
        # 提取来源
        source = soup.find('span', class_='source') or soup.find('div', class_='info')
        source = source.get_text(strip=True) if source else "未知来源"
        
        return {
            'content': content_text,
            'source': source,
            'keywords': extract_keywords(content_text)
        }
    except Exception as e:
        print(f"详情页解析失败: {str(e)[:50]}")
        return None

def extract_keywords(text, top_n=5):
    """简单关键词提取（示例）"""
    from collections import Counter
    words = [word for word in jieba.cut(text) if len(word) >= 2]
    return ', '.join([w for w, _ in Counter(words).most_common(top_n)])

def main():
    # 配置输出
    output_dir = './news_data/河北新闻/'
    os.makedirs(output_dir, exist_ok=True)
    
    all_news = []
    
    # 获取列表页数据
    print("正在抓取新闻列表...")
    news_list = parse_news_list(BASE_URL)
    print(f"发现 {len(news_list)} 条有效新闻")
    
    # 处理详情页
    for idx, news in enumerate(news_list[:10]):  # 测试时限制数量
        print(f"正在处理 {idx+1}/{len(news_list)}: {news['title']}")
        detail = parse_news_detail(news['url'])
        if detail:
            all_news.append({**news, **detail})
        
        # 动态延迟（根据服务器响应调整）
        time.sleep(random.uniform(1.5, 3.5))
    
    # 保存结果
    if all_news:
        df = pd.DataFrame(all_news)
        df = df[['title', 'time', 'source', 'keywords', 'content', 'url']]
        output_path = os.path.join(output_dir, f'hebei_news_{time.strftime("%Y%m%d%H%M")}.csv')
        df.to_csv(output_path, index=False, encoding='utf-8-sig')
        print(f"成功保存 {len(df)} 条数据到 {output_path}")
    else:
        print("未获取到有效数据")

if __name__ == '__main__':
    # 安装中文分词库（可选）
    try:
        import jieba
    except ImportError:
        print("如需使用关键词提取功能，请先安装jieba：pip install jieba")
    
    main()

正在抓取新闻列表...


Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache


发现 29 条有效新闻
正在处理 1/29: 习近平对精神文明建设工作作出重要指示


Loading model cost 0.633 seconds.
Prefix dict has been built successfully.


正在处理 2/29: 中高考临近，请收下这份考前调适“锦囊”
正在处理 3/29: 
正在处理 4/29: 202个全国文明城市(区)、60名(组)全国道德模范等受表彰
正在处理 5/29: 
正在处理 6/29: 国家医保局：推动100余项新治疗项目加快进入临床
正在处理 7/29: 神二十乘组首次出舱 为何半夜起床关门？
正在处理 8/29: "央企内推""直签保录"?当心就业不成遭欺诈
正在处理 9/29: 警惕身边的“隐形烟害”：二手烟与三手烟的健康威胁
正在处理 10/29: 202个全国文明城市（区）、60名（组）全国道德模范等受表彰
成功保存 10 条数据到 ./news_data/河北新闻/hebei_news_202505232254.csv
